In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import os

2022-03-30 14:07:04.401160: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-30 14:07:04.401183: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
path_of_saved_model = Path("./trained_model.h5")
model = tf.keras.models.load_model(path_of_saved_model)

2022-03-30 14:07:24.006177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 14:07:24.006491: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-30 14:07:24.006595: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-03-30 14:07:24.006672: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-03-30 14:07:24.006747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [3]:
path_of_saved_model

PosixPath('trained_model.h5')

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [5]:
tflite_model = converter.convert()

2022-03-30 14:13:37.778967: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmp330lqe99/assets


2022-03-30 14:13:42.745739: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-03-30 14:13:42.745800: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-03-30 14:13:42.747791: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp330lqe99
2022-03-30 14:13:42.757359: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-03-30 14:13:42.757431: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmp330lqe99
2022-03-30 14:13:42.785863: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-30 14:13:43.096430: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmp330lqe99
2022-03-30 14:13:43.188491: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 440710 

In [6]:
tflite_model_dir = Path("./tflite_model_dir")
tflite_model_dir.mkdir(exist_ok=True, parents=True)

In [7]:
tflite_model_file = tflite_model_dir/"vgg16_model.tflite"
tflite_model_file.write_bytes(tflite_model)

59067400

In [12]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

tflite_model_quant_file = tflite_model_dir/"vgg16_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpzt5hf9cu/assets


INFO:tensorflow:Assets written to: /tmp/tmpzt5hf9cu/assets
2022-03-30 14:19:54.451139: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-03-30 14:19:54.451178: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-03-30 14:19:54.451521: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpzt5hf9cu
2022-03-30 14:19:54.457334: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-03-30 14:19:54.457371: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpzt5hf9cu
2022-03-30 14:19:54.473372: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-30 14:19:54.597840: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpzt5hf9cu
2022-03-30 14:19:54.632495: I tensorflow/cc/saved_model/loader.cc:301] SavedModel

14870976

In [14]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()

In [15]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

In [16]:
import matplotlib.pyplot as plt
test_img = plt.imread("./data/unzip_data_dir/hymenoptera_data/val/ants/800px-Meat_eater_ant_qeen_excavating_hole.jpg")

In [17]:
test_img.shape

(534, 800, 3)

In [24]:
interpreter.get_input_details()

[{'name': 'serving_default_input_2:0',
  'index': 0,
  'shape': array([  1, 224, 224,   3], dtype=int32),
  'shape_signature': array([ -1, 224, 224,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [21]:
interpreter.get_input_details()[0]['shape']

array([  1, 224, 224,   3], dtype=int32)

In [23]:
resized_img = tf.image.resize(test_img, (224, 224))
input_data = tf.expand_dims(resized_img, axis=0)
input_data.shape

TensorShape([1, 224, 224, 3])

In [25]:
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [26]:
interpreter.set_tensor(input_index, input_data)
interpreter.invoke()

pred = interpreter.get_tensor(output_index)

In [27]:
pred

array([[1., 0.]], dtype=float32)

In [28]:
label_map = {0: 'ants', 1: 'bees'}

In [29]:
argmax = tf.argmax(pred[0]).numpy()
label_map[argmax]

'ants'

In [30]:
input_index = interpreter_quant.get_input_details()[0]["index"]
output_index = interpreter_quant.get_output_details()[0]["index"]

interpreter_quant.set_tensor(input_index, input_data)
interpreter_quant.invoke()
pred = interpreter_quant.get_tensor(output_index)

argmax = tf.argmax(pred[0]).numpy()
label_map[argmax]

'ants'